@author Gediyon M. Girma

TD(n) or n-step SARSA controllers of TD(2), TD(3), and TD(4) and compare thier performances for MountainCar environment.

In [ ]:
import gym
import time
import itertools
import random
import numpy as np
env = gym.make('MountainCar-v0')

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
bins = (30, 30)

def discretize_observation(observation):
  """
  Discretizes the observation space.

  Args:
    observation: The observation to discretize.

  Returns:
    The discretized observation.
  """


  low = env.observation_space.low
  high = env.observation_space.high

  return tuple(np.digitize(observation[i], np.linspace(low[i], high[i], bins[i] + 1)) - 1 for i in range(2))

In [ ]:
# TD(n) n-step SARSA on-policy using epsilon-greedy policy

alpha = 0.1 #step size for incremental averaging
gamma = 1 #discount factor
n = 2 # number of steps
epsilon = 0.1

# formaulate the state space with every combination of the discritsized elements of the states
states = itertools.product(np.arange(bins[0]), np.arange(bins[1]))

policy = {}
q = {}



for state in states:
  q[state] = np.zeros(env.action_space.n) # initialize the action-value function
  policy[state] = np.full(((env.action_space.n)),epsilon/env.action_space.n) # initialize the policy
  policy[state][np.random.randint(3)] += 1 - epsilon # initialize a random policy

In [ ]:

def TD_n(n):
  print("\n")
  print("TD(",n,") n-SARSA on-policy training for MountainCar-v0")
  episodes = 5e4
  start_timer = time.time()
  episode = 1

  while episode < episodes:
    episode_tracker = []

    # reset the environment
    obs = env.reset()
    state = discretize_observation(obs) # discretize the observation
    action = np.random.choice(np.arange(env.action_space.n), p = policy[state]) # select an action

    episode_tracker.append([0,state,action])
    T = float('inf') # terminal state

    t = 0 #initialize the time-step counter t

    while True:
      if t<T:

        obs, reward, done, info = env.step(action) # taking the action
        next_state = discretize_observation(obs) # next state

        episode_tracker.append([reward, next_state])

        if done:
          T = t+1
        else:
          action = np.random.choice(np.arange(env.action_space.n), p = policy[state]) # select an action
          episode_tracker[-1].append(action)

      tau = t - n + 1 # the time step whose state estimate is being updated (can be thought of us going n steps back)

      if tau >= 0:

        G = sum([(gamma**(i-tau-1))*episode_tracker[i][0] for i in range(tau + 1, min(tau + n, T))]) # compute the return


        if tau + n < T:
          G += (gamma**n)*q[episode_tracker[tau + n][1]][episode_tracker[tau + n][2]] # compute the discounted return

        q[episode_tracker[tau][1]][episode_tracker[tau][2]] += alpha * (G - q[episode_tracker[tau][1]][episode_tracker[tau][2]]) # update the action-value function

        policy[episode_tracker[tau][1]] = np.full(((env.action_space.n)),epsilon/env.action_space.n) # update the policy
        best_action = np.argmax(q[episode_tracker[tau][1]])
        policy[episode_tracker[tau][1]][best_action] += 1-epsilon

      if tau == T-1:  # when the update time reaches terminal state, break
        break

      state = next_state  # update the state

      t += 1 # updating the time- step

    episode += 1
    if episode % 10000 == 0:
      end_timer = time.time()
      timer = end_timer - start_timer
      elapsed_time_struct = time.gmtime(timer)
      formatted_time = time.strftime("%H:%M:%S", elapsed_time_struct)
      print("Episode: ",episode, " time: ", formatted_time)

    if episode == episodes:
      print("done!")


  #test
  steps_to_solution = []

  for j in range(100):
    obs = env.reset()
    state = discretize_observation(obs)
    for i in range(1,50000):
      action = np.argmax(policy[state])
      # print("step",i, "reward: ", reward," state: ", state," action=",action)
      obs, reward, done, info = env.step(action)
      state = discretize_observation(obs)

      if done:
        #print ("done")
        steps_to_solution.append(i)
        break
  print(steps_to_solution)
  avg_step = np.mean(steps_to_solution)
  print("Average steps to solution per 100 episodes: ",avg_step)

  # ipythondisplay.clear_output(wait=True)
  env.close()
  return

In [ ]:
for n in [2,3,4]:
  TD_n(n)



TD( 2 ) n-SARSA on-policy training for MountainCar-v0
Episode:  10000  time:  00:06:08
Episode:  20000  time:  00:11:44
Episode:  30000  time:  00:17:12
Episode:  40000  time:  00:22:38
Episode:  50000  time:  00:28:12
done!
[148, 149, 156, 153, 119, 150, 148, 155, 150, 152, 153, 128, 150, 128, 151, 150, 151, 148, 147, 193, 128, 149, 200, 148, 149, 200, 150, 150, 193, 149, 152, 119, 150, 149, 128, 172, 151, 128, 150, 152, 200, 152, 152, 153, 150, 123, 153, 152, 146, 123, 149, 150, 118, 123, 151, 149, 124, 147, 151, 117, 150, 151, 123, 155, 148, 152, 200, 117, 125, 150, 117, 150, 150, 148, 200, 117, 127, 117, 125, 149, 154, 152, 152, 129, 150, 150, 200, 149, 153, 117, 146, 151, 147, 150, 152, 124, 119, 153, 149, 151]
Average steps to solution per 100 episodes:  147.48


TD( 3 ) n-SARSA on-policy training for MountainCar-v0
Episode:  10000  time:  00:05:31
Episode:  20000  time:  00:10:56
Episode:  30000  time:  00:16:31
Episode:  40000  time:  00:21:57
Episode:  50000  time:  00:27:22